In [1]:
import pandas as pd
import requests
from tqdm import tqdm_gui, tqdm
import time
import os
import sys


pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
# Coinbase coins
r = requests.get('https://api.exchange.coinbase.com/products')
coins = pd.DataFrame.from_records(r.json())
coins = coins[(coins['quote_currency'].isin(['USD'])) & (coins['status'] == 'online')]
coins

,id,base_currency,quote_currency,quote_increment,base_increment,display_name,min_market_funds,margin_enabled,post_only,limit_only,cancel_only,status,status_message,trading_disabled,fx_stablecoin,max_slippage_percentage,auction_mode,high_bid_limit_percentage
3,USDT-USD,USDT,USD,0.00001,0.01,USDT-USD,1,False,False,False,False,online,,False,True,0.01000000,False,0.03000000
4,MINA-USD,MINA,USD,0.001,0.001,MINA-USD,1,False,False,False,False,online,,False,False,0.03000000,False,
8,AMP-USD,AMP,USD,0.00001,1,AMP-USD,1,False,False,False,False,online,,False,False,0.03000000,False,
11,MKR-USD,MKR,USD,0.01,0.000001,MKR-USD,1,False,False,False,False,online,,False,False,0.03000000,False,
13,MAGIC-USD,MAGIC,USD,0.0001,0.01,MAGIC-USD,1,False,False,False,False,online,,False,False,0.03000000,False,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
608,COVAL-USD,COVAL,USD,0.00001,1,COVAL-USD,1,False,False,False,False,online,,False,False,0.03000000,False,
612,YFI-USD,YFI,USD,0.01,0.000001,YFI-USD,1,False,False,False,False,online,,False,False,0.03000000,False,
614,ARB-USD,ARB,USD,0.0001,0.01,ARB-USD,1,False,False,False,False,online,,False,False,0.03000000,False,
619,COMP-USD,COMP,USD,0.01,0.001,COMP-USD,1,False,False,False,False,online,,False,False,0.03000000,False,


In [3]:
# Market Cap
url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest?limit=5000'
headers = {'X-CMC_PRO_API_KEY' : 'a5b199a5-0641-4d03-8cf8-2df1e19e52ef'}
r = requests.get(url, headers=headers)
coins_info = pd.DataFrame(r.json()['data'])[['name', 'symbol', 'date_added', 'tags', 'quote']]
coins_info['market_cap'] = coins_info['quote'].apply(lambda x: x['USD']['market_cap'])
coins_info = coins_info.drop(columns=['quote'])
coins_info

,name,symbol,date_added,tags,market_cap
0,Bitcoin,BTC,2010-07-13T00:00:00.000Z,"[mineable, pow, sha-256, store-of-value, state...",1277315288632.126
1,Ethereum,ETH,2015-08-07T00:00:00.000Z,"[pos, smart-contracts, ethereum-ecosystem, coi...",377101665650.718
2,Tether USDt,USDT,2015-02-25T00:00:00.000Z,"[payments, stablecoin, asset-backed-stablecoin...",109751312690.366
3,BNB,BNB,2017-07-25T00:00:00.000Z,"[marketplace, centralized-exchange, payments, ...",85288696901.609
4,Solana,SOL,2020-04-10T00:00:00.000Z,"[pos, platform, solana-ecosystem, cms-holdings...",67200489802.290
...,...,...,...,...,...
4995,MIR Token,MIR,2023-02-19T22:38:48.000Z,[],0.000
4996,SMARTSET TOKEN,SST,2023-12-15T00:40:57.000Z,[ethereum-ecosystem],0.000
4997,AmazingTeamDAO,AMAZINGTEAM,2022-04-18T09:33:01.000Z,[],0.000
4998,DackieSwap,DACKIE,2023-06-15T06:11:02.000Z,[],0.000


In [4]:
coins['symbol'] = coins['base_currency'].values
coins = coins.merge(coins_info, how='inner', on=['symbol'])
coins

,id,base_currency,quote_currency,quote_increment,base_increment,display_name,min_market_funds,margin_enabled,post_only,limit_only,...,trading_disabled,fx_stablecoin,max_slippage_percentage,auction_mode,high_bid_limit_percentage,symbol,name,date_added,tags,market_cap
0,USDT-USD,USDT,USD,0.00001,0.01,USDT-USD,1,False,False,False,...,False,True,0.01000000,False,0.03000000,USDT,Tether USDt,2015-02-25T00:00:00.000Z,"[payments, stablecoin, asset-backed-stablecoin...",109751312690.366
1,MINA-USD,MINA,USD,0.001,0.001,MINA-USD,1,False,False,False,...,False,False,0.03000000,False,,MINA,Mina,2021-03-02T00:00:00.000Z,"[pos, zero-knowledge-proofs, staking, coinbase...",938193757.252
2,AMP-USD,AMP,USD,0.00001,1,AMP-USD,1,False,False,False,...,False,False,0.03000000,False,,AMP,Amp,2020-09-08T00:00:00.000Z,"[sec-security-token, alleged-sec-securities]",324364835.859
3,MKR-USD,MKR,USD,0.01,0.000001,MKR-USD,1,False,False,False,...,False,False,0.03000000,False,,MKR,Maker,2017-01-29T00:00:00.000Z,"[store-of-value, defi, dao, polychain-capital-...",2868636834.433
4,MAGIC-USD,MAGIC,USD,0.0001,0.01,MAGIC-USD,1,False,False,False,...,False,False,0.03000000,False,,MAGIC,Treasure,2021-11-18T11:20:24.000Z,"[gaming, metaverse, arbitrum-ecosytem]",212606369.262
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,COVAL-USD,COVAL,USD,0.00001,1,COVAL-USD,1,False,False,False,...,False,False,0.03000000,False,,COVAL,Circuits of Value,2015-01-23T00:00:00.000Z,"[collectibles-nfts, defi, bitcoin-ecosystem]",45995878.644
267,YFI-USD,YFI,USD,0.01,0.000001,YFI-USD,1,False,False,False,...,False,False,0.03000000,False,,YFI,yearn.finance,2020-07-18T00:00:00.000Z,"[defi, dao, yield-farming, yield-aggregator, y...",236418549.551
268,ARB-USD,ARB,USD,0.0001,0.01,ARB-USD,1,False,False,False,...,False,False,0.03000000,False,,ARB,Arbitrum,2023-03-23T12:50:11.000Z,"[scaling, dao, dapp, ethereum-ecosystem, polyc...",3179329072.984
269,COMP-USD,COMP,USD,0.01,0.001,COMP-USD,1,False,False,False,...,False,False,0.03000000,False,,COMP,Compound,2020-06-16T00:00:00.000Z,"[defi, dao, yield-farming, coinbase-ventures-p...",475745219.175


In [5]:
sample = coins.sort_values(by=['market_cap'], ascending=False).head(100)
sample = sample.reset_index(drop=True)
sample

,id,base_currency,quote_currency,quote_increment,base_increment,display_name,min_market_funds,margin_enabled,post_only,limit_only,...,trading_disabled,fx_stablecoin,max_slippage_percentage,auction_mode,high_bid_limit_percentage,symbol,name,date_added,tags,market_cap
0,BTC-USD,BTC,USD,0.01,0.00000001,BTC-USD,1,False,False,False,...,False,False,0.02000000,False,,BTC,Bitcoin,2010-07-13T00:00:00.000Z,"[mineable, pow, sha-256, store-of-value, state...",1277315288632.126
1,ETH-USD,ETH,USD,0.01,0.00000001,ETH-USD,1,False,False,False,...,False,False,0.02000000,False,,ETH,Ethereum,2015-08-07T00:00:00.000Z,"[pos, smart-contracts, ethereum-ecosystem, coi...",377101665650.718
2,USDT-USD,USDT,USD,0.00001,0.01,USDT-USD,1,False,False,False,...,False,True,0.01000000,False,0.03000000,USDT,Tether USDt,2015-02-25T00:00:00.000Z,"[payments, stablecoin, asset-backed-stablecoin...",109751312690.366
3,SOL-USD,SOL,USD,0.01,0.00000001,SOL-USD,1,False,False,False,...,False,False,0.03000000,False,,SOL,Solana,2020-04-10T00:00:00.000Z,"[pos, platform, solana-ecosystem, cms-holdings...",67200489802.290
4,XRP-USD,XRP,USD,0.0001,0.000001,XRP-USD,1,False,False,False,...,False,False,0.05000000,False,,XRP,XRP,2013-08-04T00:00:00.000Z,"[medium-of-exchange, enterprise-solutions, arr...",28928065315.656
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,LRC-USD,LRC,USD,0.0001,0.000001,LRC-USD,1,False,False,False,...,False,False,0.03000000,False,,LRC,Loopring,2017-08-30T00:00:00.000Z,"[marketplace, decentralized-exchange-dex-token...",372160719.588
96,DASH-USD,DASH,USD,0.01,0.001,DASH-USD,1,False,False,False,...,False,False,0.03000000,False,,DASH,Dash,2014-02-14T00:00:00.000Z,"[mineable, hybrid-pow-pos, x11, medium-of-exch...",364539685.779
97,T-USD,T,USD,0.00001,1,T-USD,1,False,False,False,...,False,False,0.03000000,False,,T,Threshold,2022-01-31T08:26:28.000Z,[bitcoin-ecosystem],352494847.332
98,ZETA-USD,ZETA,USD,0.0001,0.1,ZETA-USD,1,False,False,False,...,False,False,0.03000000,False,,ZETA,ZetaChain,2022-08-05T02:45:23.000Z,"[interoperability, smart-contracts, cross-chai...",349493621.476


In [6]:
# Get all product volume
r = requests.get('https://api.exchange.coinbase.com/products/volume-summary')
volume = pd.DataFrame.from_records(r.json())
volume

,id,base_currency,quote_currency,display_name,market_types,spot_volume_24hour,spot_volume_30day,rfq_volume_24hour,rfq_volume_30day,conversion_volume_24hour,conversion_volume_30day
0,ETH-TONE,ETH,TONE,ETH-TONE,[rfq],,,0,0,0,0
1,DOT-NEAR,DOT,NEAR,DOT-NEAR,[rfq],,,211.858069,14586.379100,0,0
2,SWFTC-SYLO,SWFTC,SYLO,SWFTC-SYLO,[rfq],,,0,0,0,0
3,ETH-RLC,ETH,RLC,ETH-RLC,[rfq],,,0,0.236170,0,0
4,SHIB-USD,SHIB,USD,SHIB-USD,"[spot, rfq]",2611090554764,104741370938707,38196249096.318230,1479854214405.495850,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2479,CVX-USDC,CVX,USDC,CVX-USDC,[rfq],,,0,61.244548,0,0
2480,PUNDIX-USD,PUNDIX,USD,PUNDIX-USD,"[spot, rfq]",2955556.7,29124685.05,8260.969499,79918.863663,0,0
2481,ACS-LTC,ACS,LTC,ACS-LTC,[rfq],,,2810.640662,92641.035980,0,0
2482,ROSE-USDT,ROSE,USDT,ROSE-USDT,"[spot, rfq]",1175355.7,53207242.6,0,0,0,0


In [7]:
# Collect categories
url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/categories'
headers = {'X-CMC_PRO_API_KEY' : 'a5b199a5-0641-4d03-8cf8-2df1e19e52ef'}
r = requests.get(url, headers=headers)
categories = pd.DataFrame(r.json()['data'])
categories['category_id'] = categories['id'].values
categories

,id,name,title,description,num_tokens,avg_price_change,market_cap,market_cap_change,volume,volume_change,last_updated,category_id
0,6617dd1bd0384836b9c7bdf3,TokenFi Launchpad,TokenFi Launchpad,TokenFi Launchpad,4,17.495,1697258491.150,9.882,229840527.421,14.349,2024-04-11T12:53:04.936Z,6617dd1bd0384836b9c7bdf3
1,6604f2405726cb6104472fdb,Presale Memes,Presale Memes,Presale Memes,3,-1.680,197907676.670,-5.094,93783877.534,-14.822,2024-03-28T04:30:07.129Z,6604f2405726cb6104472fdb
2,66039e3c6253043a6a580e65,Move VM (MVM),Move VM (MVM),Move VM (MVM),2,3.290,6102170727.570,3.310,362344399.952,-32.187,2024-03-27T04:19:33.033Z,66039e3c6253043a6a580e65
3,65fafbaccd69e615d550867e,Atomicals Ecosystem,Atomicals Ecosystem,Atomicals Ecosystem,8,5.019,0.000,0.000,1196364.025,1.318,2024-03-20T16:19:23.093Z,65fafbaccd69e615d550867e
4,65f23191e6c934565751ce16,DePIN,DePIN,DePIN,134,4.300,35519017647.610,3.635,1454951902.358,-0.164,2024-03-13T23:07:23.483Z,65f23191e6c934565751ce16
...,...,...,...,...,...,...,...,...,...,...,...,...
229,60308028d2088f200c58a005,BNB Chain Ecosystem,Top BNB Chain Tokens by Market Capitalization,BNB Chain dual-chain architecture will empower...,1997,1.999,263436963395.930,1.186,53144787814.504,1646.696,2023-08-09T13:59:47.270Z,60308028d2088f200c58a005
230,60291fa0db1be76c46298e83,NFTs & Collectibles,Top NFTs & Collectibles Tokens by Market Capit...,A NFT (non-fungible token) is a special crypto...,772,2.681,43473075340.940,1.649,2670081428.573,1441.944,2023-08-09T13:45:05.229Z,60291fa0db1be76c46298e83
231,601cf8d2d8fd860e4ea5d96f,Polkadot Ecosystem,Top Polkadot Tokens by Market Capitalization,Polkadot is an open-source sharding multichain...,109,2.492,26859771608.100,2.408,789030165.279,18.026,2023-08-09T14:00:10.601Z,601cf8d2d8fd860e4ea5d96f
232,5fb62da404d1dd4c73744883,Storage,Top Storage Tokens by Market Capitalization,In blockchain-based decentralized cloud storag...,53,4.295,17011885866.130,4.569,691672697.369,5.251,2023-08-09T13:48:02.585Z,5fb62da404d1dd4c73744883


In [9]:
selected_cats = categories.sort_values(by=['num_tokens'], ascending=False).head(100)

frames = []
for _id in tqdm(selected_cats['id'].unique()):
    url = f'https://pro-api.coinmarketcap.com/v1/cryptocurrency/category?id={_id}&limit=100'
    headers = {'X-CMC_PRO_API_KEY' : 'a5b199a5-0641-4d03-8cf8-2df1e19e52ef'}
    r = requests.get(url, headers=headers)
    try:
        frame = pd.DataFrame.from_records(r.json()['data']['coins'])
        frame['category_id'] = _id
        frames.append(frame)
        time.sleep(.5)
    except:
        continue

category_mapping = pd.concat(frames, ignore_index=True)
category_mapping = category_mapping[['id', 'symbol', 'category_id']]
category_mapping = category_mapping.merge(categories[['category_id', 'name', 'description']], how='left', on=['category_id'])
category_mapping

100%|██████████| 100/100 [01:16<00:00,  1.30it/s]
/var/folders/b2/17vn_90d5t93nhh5__zt6tb80000gn/T/ipykernel_37335/1230674035.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  category_mapping = pd.concat(frames, ignore_index=True)


,id,symbol,category_id,name,description
0,74,DOGE,6051a82566fc1b42617d6dc6,Memes,Memes
1,5994,SHIB,6051a82566fc1b42617d6dc6,Memes,Memes
2,28752,WIF,6051a82566fc1b42617d6dc6,Memes,Memes
3,24478,PEPE,6051a82566fc1b42617d6dc6,Memes,Memes
4,10804,FLOKI,6051a82566fc1b42617d6dc6,Memes,Memes
...,...,...,...,...,...
3596,2274,MDS,6051a80b66fc1b42617d6da4,Health,Health
3597,6237,TMED,6051a80b66fc1b42617d6da4,Health,Health
3598,1529,KUSH,6051a80b66fc1b42617d6da4,Health,Health
3599,2497,MTN,6051a80b66fc1b42617d6da4,Health,Health


In [10]:
category_mapping.to_csv('crypto_categories.csv', index=False)

In [ ]:
# res = coins.merge(volume[['id', 'rfq_volume_30day']], how='left', on=['id'])
# res.sort_values(by = ['rfq_volume_30day'], ascending=False).head(30)

In [ ]:
url = 'https://api.pro.coinbase.com/products/BTC-USD/candles?start=2023-08-01T12:00:00&end=2024-04-20T12:00:00&granularity=86400'
r = requests.get(url)
frame = pd.DataFrame(r.json(), columns = ['date', 'low', 'high', 'open', 'close', 'volume'])
frame['date'] = pd.to_datetime(frame['date'], unit='s')
frame

,date,low,high,open,close,volume
0,2024-04-20,63121.15,65463.08,63841.96,65058.44,4191.250006
1,2024-04-19,59573.32,65498.99,63514.84,63848.37,22838.079342
2,2024-04-18,60816.07,64210.00,61275.73,63513.85,15246.985398
3,2024-04-17,59648.33,64524.14,63814.94,61279.36,21811.867049
4,2024-04-16,61645.09,64400.00,63452.18,63814.94,17710.282132
...,...,...,...,...,...,...
258,2023-08-06,28955.00,29163.24,29047.33,29040.01,2461.640121
259,2023-08-05,28946.17,29111.87,29077.04,29047.12,2490.847105
260,2023-08-04,28747.10,29312.28,29174.84,29076.48,7291.146877
261,2023-08-03,28938.93,29399.97,29163.87,29174.18,8920.014777
